In [1]:
!pip install anthropic-haystack

In [2]:
import os
from getpass import getpass
from typing import List

from haystack_experimental.components.agents import Agent
from haystack_experimental.components.tools.tool_context import ToolContext

from haystack.tools import Tool, tool
from haystack.dataclasses import ChatMessage, Document

from github_components.repo_viewer import GithubRepositoryViewer

In [3]:
from agent_prompts import issue_prompt, repo_viewer_prompt, repo_viewer_schema

In [4]:
def view_repository(ctx:  ToolContext, repo: str, path: str = "") -> str:
    viewer = GithubRepositoryViewer(raise_on_failure=False)

    result = viewer.run(path=path, repo=repo)

    result_str = ""
    for document in result["documents"]:
        if document.meta["type"] in ["file", "dir", "error"]:
            result_str += document.content + "\n"
        else:
            result_str += f"File Content for {document.meta['path']}\n\n"
            result_str += document.content

    if len(result_str) > 150_000:
        result_str = result_str[:150_000] + "...(large file can't be fully displayed)"

    if output_docs := ctx.get_output("documents"):
        output_docs.extend(result["documents"])
        ctx.set_output(key="documents", value=output_docs)
    else:
        ctx.set_output(key="documents", value=result["documents"])

    return result_str

In [5]:
repo_viewer_tool = Tool(
    name="view_repository",
    description=repo_viewer_prompt,
    parameters=repo_viewer_schema,
    function=view_repository,
)

In [6]:
@tool
def create_comment(comment: str) -> str:
    """
    Use this to create a Github comment once you finished your exploration.
    """
    return comment


In [11]:
os.environ["LLM_API_KEY"] = getpass("Anthropic Key: ")

In [12]:
agent = Agent(
    model="anthropic:claude-3-5-sonnet-latest",
    system_prompt=issue_prompt,
    tools=[repo_viewer_tool, create_comment],
    generation_kwargs={"max_tokens": 8000},
    handoff="create_comment",
    output_variables={"documents": List[Document]}
)

In [13]:
issue_url = ChatMessage.from_user("https://github.com/deepset-ai/haystack/issues/8769")

In [14]:
result = agent.run(messages=[issue_url])

In [22]:
from IPython.display import display, Markdown
display(Markdown(result["messages"][-1].tool_call_result.result))

Hi @deepansha16! 👋 

You raise a good point about the Python version compatibility between haystack and haystack-experimental. Let me explain why we have this difference:

1. The main haystack package supports Python 3.8+ to maintain broader compatibility, especially for users who might be constrained to older Python versions.

2. However, for haystack-experimental, we made a deliberate choice to require Python 3.9+ because:
   - Python 3.9 introduced several important typing improvements that are particularly useful for experimental features where we want to leverage the latest type hints
   - As an experimental package, we have more flexibility to require newer Python versions since it's not meant for production use
   - This allows us to use modern Python features without needing to maintain compatibility with older versions

While it would be technically possible to support Python 3.8 in haystack-experimental, this would require:
- Additional compatibility code
- More complex typing workarounds
- Potentially giving up some modern Python features we're using in the experimental components

Given that Python 3.8 is approaching its end of life (scheduled for October 2024), we believe it's better to encourage users to move to newer Python versions, especially for experimental features.

If you're constrained to Python 3.8, I would recommend:
1. Using only the main haystack package without the experimental features
2. Or if possible, upgrading to Python 3.9+ to take advantage of the experimental components

Let me know if you have any other questions!